In [ ]:
from pathlib import Path
import numpy as np

In [ ]:
from split_dataset import SplitDataset
from fimpy.pipeline.alignment import align_2p_volume

In [ ]:
master = Path(r"Z:\Anja & Hagar")
all_fish = list(master.glob("*f[0-9]*"))
fish_dir = all_fish[-2]
print(all_fish)

In [ ]:
source_dir = "anatomy\original"
source_dir = "original"

frames_per_plane = None
aligned_dir = "aligned"
force_realign = False
force_remake_tifs = False
force_remake_corr = False

In [ ]:
force_realign = False
for f in [fish_dir]:
    print(f)
    if not(f/'aligned').exists() or force_realign:
        align_2p_volume(SplitDataset(f / source_dir), output_dir=str(f), across_planes=False)
    else:
        print("Already aligned")

In [ ]:
align_2p_volume(SplitDataset(fish_dir / source_dir), output_dir=str(fish_dir), across_planes=False)

In [ ]:
ds = SplitDataset(fish_dir / "aligned")
stack = ds[:, :, :, :]

In [ ]:
print(stack.max(), stack.mean())
mean = stack.mean(0)
print(np.shape(mean))
skio.imsave(str(fish_dir / "anatomy.tif"), mean)

In [ ]:
ds = 0
stack = 0

In [ ]:
ds = SplitDataset(fish_dir / source_dir)
stack = ds[:, :, :, :]
print(stack.max(), stack.mean())
mean = stack.mean(0)
print(np.shape(mean))
skio.imsave(str(fish_dir / "anatomy_original.tif"), mean)

In [ ]:
mean = skio.imread(str(fish_dir /  "anatomy.tif"))

In [ ]:
align_2p_volume(SplitDataset(fish_dir / source_dir), output_dir=str(fish_dir), reference=mean, across_planes=False)
ds = SplitDataset(fish_dir / "aligned")
stack = ds[:, :, :, :]
print(stack.max(), stack.mean())
mean = stack.mean(0)
print(np.shape(mean))
skio.imsave(str(fish_dir / "anatomy2.tif"), mean.astype(np.uint16))
ds = 0
stack = 0

In [ ]:
#anatomy = skio.imread(str(fish_dir / "anatomy1.tif"))
#anatomy2 = anatomy.T
#print(np.shape(anatomy2))

In [ ]:
force_realign = True
force_remake_tifs = False 

for i in range(2, 5):
    
    anatomy_file = "anatomy" + str(i) + ".tif"
    
    # Create anatomy tif file:
    if not (fish_dir / anatomy_file).exists() or force_remake_tifs:
        print("Saving anatomy...")
        ds = SplitDataset(fish_dir / "aligned")
        stack = ds[:, :, :, :]
        print(stack.max())
        mean = stack.mean(0)

        skio.imsave(str(fish_dir / anatomy_file), mean.astype(np.uint16))
        stack = 0
        
    # Align:
    if not (fish_dir / aligned_dir).exists() or force_realign:
        print("Aligning...")

        anatomy = skio.imread(str(fish_dir / anatomy_file))
        #anatomy = anatomy.T
        align_2p_volume(SplitDataset(fish_dir / source_dir), output_dir=str(fish_dir), reference=anatomy, across_planes=False)
    else:
        print("Already aligned")

        

In [ ]:
if not (fish_dir / "corrmap.tif").exists():
    print("Saving correlation map")
    ds = SplitDataset(fish_dir / "aligned")
    corr_map = correlation_map(ds)[:, :, :]
    skio.imsave(str(fish_dir / "corrmap.tif"), corr_map.astype(np.float))
    
ds = 0

In [ ]:
import flammkuchen as fl
from fimpy.pipeline.roi_extraction import extract_traces

In [ ]:
# Extracting traces: 
# master = Path(r"C:\Users\lavian\Desktop\data\e0040_V13")
master = Path(r"\\Funes\Shared\experiments\E0040_motions_cardinal\v13_cw_ccw\2p\habenula\fixed")
master = Path(r"C:\Users\lavian\Desktop\data\e0040_V13\rf\fixed")
all_fish = list(master.glob("*f[0-9]*"))
fish_dir = all_fish[5]
print(fish_dir)

In [ ]:
dir_rois = str(fish_dir/'merged_rois.h5')
dir_aligned = str(fish_dir / "aligned")
aligned = SplitDataset(dir_aligned)

In [ ]:
rois_arr = fl.load(dir_rois)['stack'] 
rois_arr = rois_arr.astype(int) - 1
traces = extract_traces(aligned, rois_arr)
np.shape(rois_arr)

In [ ]:
print(dir_rois)